In [2]:
from hyperopt import fmin, tpe, hp
best = fmin(fn=lambda x: x ** 2,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100)
print(best)

{'x': 0.078817034006613}


In [4]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(x):
    return {'loss': x ** 2, 'status': STATUS_OK }

best = fmin(objective,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100)

print(best)

{'x': -0.03660403483276048}


In [5]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(x):
    return {
        'loss': x ** 2,
        'status': STATUS_OK,
        # -- store other results like this
        'eval_time': time.time(),
        'other_stuff': {'type': None, 'value': [0, 1, 2]},
        # -- attachments are handled differently
        'attachments':
            {'time_module': pickle.dumps(time.time)}
        }
trials = Trials()
best = fmin(objective,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100,
    trials=trials)

print(best)

{'x': -0.013505638089387692}


In [6]:
msg = trials.trial_attachments(trials.trials[5])['time_module']
time_module = pickle.loads(msg)

In [10]:
from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing
from sklearn.datasets import load_iris
from hyperopt import tpe
import numpy as np

# Download the data and split into training and test sets

iris = load_iris()

X = iris.data
y = iris.target

test_size = int(0.2 * len(y))
np.random.seed(13)
indices = np.random.permutation(len(X))
X_train = X[indices[:-test_size]]
y_train = y[indices[:-test_size]]
X_test = X[indices[-test_size:]]
y_test = y[indices[-test_size:]]

# Instantiate a HyperoptEstimator with the search space and number of evaluations

estim = HyperoptEstimator(classifier=any_classifier('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim.fit(X_train, y_train)

# Show the results

print(estim.score(X_test, y_test))

print( estim.best_model() )

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: 

0.7666666666666667
{'learner': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=8, p=3.0,
           weights='uniform'), 'preprocs': (StandardScaler(copy=True, with_mean=False, with_std=True),), 'ex_preprocs': ()}


In [23]:
from hpsklearn import HyperoptEstimator, any_regressor, any_preprocessing
from sklearn.model_selection import train_test_split
from hyperopt import tpe
import pandas as pd
import numpy as np

# Download the data and split into training and test sets
property_assess = pd.read_csv('assessment_per_capita.csv', dtype={'suite': str})
ml = property_assess[['value', 'nb_id', 'garage', 'zoning', 'lot_size', 'year_built', 'crime_per_capita']]
dummy_cols = ['nb_id', 'garage', 'zoning']
df = pd.get_dummies(ml, columns=dummy_cols, drop_first=True)

X_large = df.drop('value', axis=1).values
y_large = df['value'].values

X = X_large[:1000]
y = y_large[:1000]

y.reshape(-1,1)

print(type(X))

<class 'numpy.ndarray'>


In [24]:
col_mean = np.nanmean(X, axis=0)

#Find indicies that you need to replace
inds = np.where(np.isnan(X))

#Place column means in the indices. Align the arrays using take
X[inds] = np.take(col_mean, inds[1])

print(np.nan in X)
print(X)

False
[[5.57000000e+02 1.96400000e+03 1.29259694e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.66000000e+02 1.96200000e+03 1.29259694e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.86000000e+02 1.96800000e+03 1.29259694e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [4.82000000e+02 1.99400000e+03 2.23463687e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.29000000e+02 1.99600000e+03 2.23463687e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.69000000e+02 2.00000000e+03 2.23463687e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# Instantiate a HyperoptEstimator with the search space and number of evaluations

estim = HyperoptEstimator(regressor=any_regressor('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim.fit(X_train, y_train)

# Show the results

print(estim.score(X_test, y_test))

print( estim.best_model() )

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: 

0.5368813478652967
{'learner': GradientBoostingRegressor(alpha=0.5, criterion='friedman_mse', init=None,
             learning_rate=0.008063281537322596, loss='quantile',
             max_depth=None, max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=3, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=900,
             n_iter_no_change=None, presort='auto', random_state=0,
             subsample=0.8090846526564667, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
